In [48]:
import streamlit as st
import pandas as pd
import plotly.express as px
import sqlite3
import os

In [49]:
lista = []
pasta = "Dados"
for arquivo in os.listdir(pasta):
    nome = arquivo.replace(".csv", "")
    dir_arquivo = os.path.join(pasta, arquivo)
    try:  # Evita erros de codificação ao ler os arquivos CSV
        df = pd.read_csv(
            dir_arquivo,
            sep=";",
            encoding="utf-8",
            parse_dates=["Data"],
            dayfirst=True,
        )
    except UnicodeDecodeError:
        df = pd.read_csv(
            dir_arquivo,
            sep=";",
            encoding="ISO-8859-1",
            parse_dates=["Data"],
            dayfirst=True,
        )
    df.set_index("Data", inplace=True)

    df.rename(columns={"Consumo(kWh)": f"{nome} (kWh)"}, inplace=True)
    lista.append(df)
    df = pd.concat(lista, axis=1).fillna(0)


In [50]:
df

,Igreja (kWh),Sra Renata (kWh),Sr Bené (kWh),Sr Raul (kWh),Sr João (kWh),Sra Iraneide (kWh),Sr Matoso (kWh),Sra Fernanda (kWh),Sr Dinho (kWh),Sr Yuri (kWh)
Data,,,,,,,,,,
2024-01-08,0.0,0.0,0.0,0.000,1.790,0.0,0.000,0.0,0.0,0.000
2024-01-09,0.0,0.0,0.0,0.000,3.092,0.0,0.000,0.0,0.0,0.000
2024-01-10,0.0,0.0,0.0,0.000,2.701,0.0,0.000,0.0,0.0,0.000
2024-01-11,0.0,0.0,0.0,0.000,2.825,0.0,0.000,0.0,0.0,0.000
2024-01-12,0.0,0.0,0.0,0.000,2.327,0.0,0.000,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...
2024-06-16,0.0,0.0,0.0,2.646,3.198,0.0,3.131,0.0,0.0,2.579
2024-06-17,0.0,0.0,0.0,3.548,3.444,0.0,2.728,0.0,0.0,2.342
2024-06-18,0.0,0.0,0.0,3.284,3.212,0.0,3.038,0.0,0.0,2.467


In [51]:
conn = sqlite3.connect('consumo.db')
c = conn.cursor()

In [52]:
c.execute('''
    CREATE TABLE IF NOT EXISTS sensor_data (
        Data TEXT,
        Hora TEXT,
        Igreja REAL,
        Sr_Bené REAL,
        Sr_Dinho REAL,
        Sr_João REAL,
        Sr_Matoso REAL,
        Sr_Raul REAL,
        Sr_Yuri REAL,
        Sra_Fernanda REAL,
        Sra_Iraneide REAL,
        Sra_Renata REAL
    )
''')

In [53]:
# Salvar (commit) as mudanças
conn.commit()

In [54]:
df.to_sql('consumo', conn, if_exists='append', index=False)


155

In [55]:
c.execute('SELECT * FROM consumo LIMIT 5')

# Buscar e imprimir os resultados
rows = c.fetchall()
for row in rows:
    print(row)

(0.0, 0.0, 0.0, 0.0, 1.79, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, 0.0, 0.0, 0.0, 3.092, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, 0.0, 0.0, 0.0, 2.701, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, 0.0, 0.0, 0.0, 2.825, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, 0.0, 0.0, 0.0, 2.327, 0.0, 0.0, 0.0, 0.0, 0.0)


In [56]:
conn.close()